In [1]:
!pip install transformers
!pip install datasets
!pip install pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00


In [3]:
!pip install demucs==2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached diffq-0.2.4-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (418 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 8.5 MB/s eta 0:00:00
  Using cached julius-0.2.7.tar.gz (59 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for demucs: filename=demucs-2.0.0-py3-none-any.whl size=40306 sha256=4380d4e320c02347d1fd2e167160108b5abe062ba308e325f8e6835b75e34cbd
  Stored in directory: /root/.cache/pip/wheels/11/16/97/b13b4d5add827ec1a0ac0763a03754836d7b647a3b483d7d85
  Created wheel for julius: filename=julius-0.2.7-py3-none-any.whl size=21869 sha256=6690d8e8f52377958ca1b1e8864f4ee36cdfd8163f3bf795028358386baa1306
  Stored in directory: /root/.cache/pip/wheels/b9/b2/05/f883527ffcb7f2ead5438a2c23439aa0c881eaa9a4c80256f4
Successfully built demucs julius


In [6]:
import soundfile as sf
import torch
import librosa
import torch
import torch.nn as nn
import torchaudio
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from datasets import load_dataset
# from demucs import pretrained
from demucs.pretrained import load_pretrained
# from demucs.pretrained import demucs_unittest
from transformers import AutoTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor

In [4]:
input_audio_file = '/content/Blessings.flac'
lrc_file = '/content/Blessings - The Kid LAROI.lrc'#lyrics start at line 4(is it always...?)

In [9]:
# chunkize
from pydub import AudioSegment
import re
import os

def parse_lrc(lrc_content):
    """
    Parse the LRC file content and extract timestamps and lyrics.
    Returns a list of tuples (start_time, end_time, lyric).
    """
    timestamps = []
    lyrics = []
    for line in lrc_content.split('\n'):
        match = re.match(r'\[(\d+):(\d+\.\d+)\](.*)', line)
        if match:
            minutes, seconds, lyric = match.groups()
            time = int(minutes) * 60 * 1000 + int(float(seconds) * 1000)  # Convert to milliseconds
            timestamps.append(time)
            lyrics.append(lyric)

    # Create tuples of (start_time, end_time, lyric)
    intervals = [(timestamps[i], timestamps[i+1], lyrics[i]) for i in range(len(timestamps)-1)]
    return intervals

def split_audio_and_lyrics(audio_file, lrc_content, output_folder):
    """
    Split the audio file into chunks and create text files for each chunk with the corresponding lyrics.
    """
    audio = AudioSegment.from_file(audio_file, format="flac")
    intervals = parse_lrc(lrc_content)

    for i, (start, end, lyric) in enumerate(intervals):
        chunk = audio[start:end]
        chunk.export(f"{output_folder}/chunk_{i+1}.mp3", format="mp3")

        with open(f"{output_folder}/chunk_{i+1}.txt", 'w') as text_file:
            text_file.write(lyric)

output_folder = '/content/trial1'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

with open(lrc_file, 'r') as file:
    lrc_content = file.read()

split_audio_and_lyrics(input_audio_file, lrc_content, output_folder)

In [7]:
class ChangeSampleRate(nn.Module):
    def __init__(self, input_rate: int, output_rate: int):
        super().__init__()
        self.output_rate = output_rate
        self.input_rate = input_rate

    def forward(self, wav: torch.tensor) -> torch.tensor:
        # Only accepts 1-channel waveform input
        wav = wav.view(wav.size(0), -1)
        new_length = wav.size(-1) * self.output_rate // self.input_rate
        indices = (torch.arange(new_length) * (self.input_rate / self.output_rate))
        round_down = wav[:, indices.long()]
        round_up = wav[:, (indices.long() + 1).clamp(max=wav.size(-1) - 1)]
        output = round_down * (1. - indices.fmod(1.)).unsqueeze(0) + round_up * indices.fmod(1.).unsqueeze(0)
        return output

In [18]:
class MyWav2Vec(nn.Module):
    def __init__(self):
      super().__init__()
      self.demucs = load_pretrained('demucs')
      self.wav2vec = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
      self.wav2vec_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
      self.changesamplerate = torchaudio.transforms.Resample(44100, 16000)

    def forward(self, input_tensor):

      # remove the commented part to see what its' like with the source separation
      # #Demucs:
      # out_demucs = self.demucs(input_tensor)
      # print(out_demucs.shape)
      # #Extract voice and squeeze:
      # output_voice = out_demucs[:, 3, :, :]
      # print(output_voice.shape)
      # #transform from stereo to mono:
      output_voice_mono = torch.mean(input_tensor, dim=1)

      # change sample rate:
      audio_sr = self.changesamplerate(output_voice_mono)

      #Wav2Vec processor function:
      input_values = (audio_sr - audio_sr.mean(1)) / torch.sqrt(audio_sr.var(1) + 1e-5)
      # input_values = torch.reshape(input_values, (audio_sr.shape[1],audio_sr.shape[2]))
      # print(input_values.shape)
      #Wav2Vec:
      logits = self.wav2vec(input_values).logits
      predicted_ids = torch.argmax(logits, dim=-1)
      transcription = self.wav2vec_processor.decode(predicted_ids[0])

      return transcription


In [14]:
audio, _ = torchaudio.load("/content/trial1/chunk_11.mp3")
print(type(audio))
sr = 44100
input_tensor_model = torch.cat([audio.unsqueeze(0)])
# print(input_tensor_model.shape)
Mymodel = MyWav2Vec()
transcription = Mymodel(input_tensor_model)
print(transcription)

<class 'torch.Tensor'>
torch.Size([1, 2, 79380])
